In [9]:
import pyodbc
server = 'act4-server.database.windows.net'
database = 'act4-database'
username = 'stanford'
password = 'Hola1234'

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")

Connected to the Azure SQL Database successfully!


In [10]:
import pandas as pd

query = "SELECT * FROM SalesLT.Customer"
df = pd.read_sql(query, conn)

print(df.head())

C:\Users\david\AppData\Local\Temp\ipykernel_20964\2608267317.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   CustomerID  NameStyle Title FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.   Orlando         N.         Gee   None   
1           2      False   Mr.     Keith       None      Harris   None   
2           3      False   Ms.     Donna         F.    Carreras   None   
3           4      False   Ms.     Janet         M.       Gates   None   
4           5      False   Mr.      Lucy       None  Harrington   None   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8   
2    Advanced Bike Components  adventure-works\jillian0   
3       Modular Cycle Systems  adventure-works\jillian0   
4  Metropolitan Sports Supply      adventure-works\shu0   

                   EmailAddress         Phone  \
0  orlando0@adventure-works.com  245-555-0173   
1    keith0@adventure-works.com  170-555-0127   
2    donna0@adventure-works.com  279-555-0130   
3   

In [11]:
df.dtypes

CustomerID               int64
NameStyle                 bool
Title                   object
FirstName               object
MiddleName              object
LastName                object
Suffix                  object
CompanyName             object
SalesPerson             object
EmailAddress            object
Phone                   object
PasswordHash            object
PasswordSalt            object
rowguid                 object
ModifiedDate    datetime64[ns]
dtype: object

In [12]:
df['ModifiedDate'] = pd.to_datetime(df['ModifiedDate'])

min_date = df['ModifiedDate'].min()
df['ModifiedDateDays'] = (df['ModifiedDate'] - min_date).dt.days



In [13]:
useful_cols = ['CustomerID', 'CompanyName', 'SalesPerson', 'ModifiedDateDays']
df_clean = df[useful_cols].copy()

df_clean = pd.get_dummies(df_clean, columns=['CompanyName', 'SalesPerson'], drop_first=True)

X = df_clean.drop(columns='ModifiedDateDays')
y = df_clean['ModifiedDateDays']

In [14]:
df_clean.to_csv('cleaned_data.csv', index=False)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))


R2 Score: 0.7013708114407509
RMSE: 171.4463192754163


In [16]:
import joblib

joblib.dump(model, 'model.pkl')


['model.pkl']

In [ ]:


from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = "f7573334-1f4f-4ce7-9b75-ce1f32e6bd37",
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-9a4e997089
Deploying KeyVault with name workspackeyvault5b835795.
Deploying StorageAccount with name workspacstorage1aad7460c.
Deploying Workspace with name workspace.
Deploying AppInsights with name workspacinsights310f2769.
Deployed AppInsights with name workspacinsights310f2769. Took 36.64 seconds.
Deployed Workspace with name workspace. Took 41.87 seconds.
Registering model model


In [ ]:


from azureml.core import Workspace

ws = Workspace.get(name="workspace",
                      subscription_id = "f7573334-1f4f-4ce7-9b75-ce1f32e6bd37",
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [21]:
scorepy = """
import json
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("model")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        df = pd.DataFrame(data)
        preds = model.predict(df).tolist()
        return json.dumps(preds)
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

# Write to file
file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

virtual_env = Environment("env-for-regression")
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "scikit-learn", "joblib"]
)

inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="score.py"XX   
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name="regression-service",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)



C:\Users\david\AppData\Local\Temp\ipykernel_20964\2091083951.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [23]:
service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-23 17:33:52-06:00 Creating Container Registry if not exists..
2025-04-23 17:43:53-06:00 Registering the environment.
2025-04-23 17:43:54-06:00 Building image..
2025-04-23 17:52:22-06:00 Generating deployment configuration.
2025-04-23 17:52:24-06:00 Submitting deployment to compute..
2025-04-23 17:52:32-06:00 Checking the status of deployment regression-service..
2025-04-23 17:53:48-06:00 Checking the status of inference endpoint regression-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
import json

scoring_uri = service.scoring_uri

with open("uri.json", "w") as file:
    json.dump({"URI": [scoring_uri]}, file)

In [25]:
import pandas as pd

df_clean = pd.read_csv("cleaned_data.csv")

X = df_clean.drop(columns=["ModifiedDateDays"])

sample_data = X.iloc[[0]] 


In [ ]:
import json
import requests

data_json = json.dumps({"data": sample_data.to_dict(orient="records")})

with open("uri.json", "r") as f:
    scoring_uri = json.load(f)["URI"][0]

headers = {"Content-Type": "application/json"}
response = requests.post(scoring_uri, data=data_json, headers=headers)

if response.status_code == 200:
    print("Prediction:", json.loads(response.text))
else:
    print(f"Error: {response.status_code} - {response.text}")


Prediction: [30.99999999961682]
